### Introduction

JSON is one of the most widely used formats in the world for applications to exchange data.

Structured Outputs is a feature that ensures the model will always generate responses that adhere to your supplied [JSON Schema](https://json-schema.org/overview/what-is-jsonschema), so you don't need to worry about the model omitting a required key, or hallucinating an invalid enum value.

Some benefits of Structured Outputs include:

- Reliable type-safety: No need to validate or retry incorrectly formatted responses
- Explicit refusals: Safety-based model refusals are now programmatically detectable
- Simpler prompting: No need for strongly worded prompts to achieve consistent formatting

In addition to supporting JSON Schema in the REST API, the OpenAI SDKs for Python and JavaScript also make it easy to define object schemas using Pydantic and Zod respectively. Below, you can see how to extract information from unstructured text that conforms to a schema defined in code.

Supported models

Structured Outputs are available in our latest large language models, starting with GPT-4o:

- o3-mini-2025-1-31 and later
- o1-2024-12-17 and later
- gpt-4o-mini-2024-07-18 and later
- gpt-4o-2024-08-06 and later

Older models like gpt-4-turbo and earlier may use JSON mode instead.

In [1]:
import json
from enum import Enum
from pydantic import BaseModel
from typing import List, Optional
from helper import set_openai_key, test_openai_api, create_openai_client, print_pretty

set_openai_key()

test_openai_api()

client = create_openai_client()

API key set successfully.
Hello! How can I assist you today?


### Chain of thought

In [2]:
class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: List[Step]
    final_answer: str

In [4]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message

# If the model refuses to respond, you will get a refusal message
if (math_reasoning.refusal):
    print(math_reasoning.refusal)
else:
    print(math_reasoning.parsed)

steps=[Step(explanation='The goal is to solve for \\( x \\), which means isolating \\( x \\) on one side of the equation. The initial equation is \\( 8x + 7 = -23 \\). To start, we need to eliminate the +7 from the left side.', output='8x + 7 = -23'), Step(explanation='Subtract 7 from both sides of the equation to move it to the other side. This helps in isolating the term with \\( x \\).', output='8x + 7 - 7 = -23 - 7'), Step(explanation='Simplifying both sides after subtracting 7. On the left side, \\( +7 \\) and \\( -7 \\) cancel each other out, leaving \\( 8x \\). On the right side, \\(-23 - 7\\) simplifies to \\(-30\\).', output='8x = -30'), Step(explanation='Now, we need to get \\( x \\) by itself. Since \\( x \\) is being multiplied by 8, we should divide both sides by 8 to solve for \\( x \\).', output='\\frac{8x}{8} = \\frac{-30}{8}'), Step(explanation='Simplifying both sides. On the left side, \\( \\frac{8x}{8} = x \\). On the right side, \\( \\frac{-30}{8} \\) can be simplif

In [5]:
print_pretty(completion.dict())

{
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "audio": null,
                "content": "{\"steps\":[{\"explanation\":\"The goal is to solve for \\\\( x \\\\), which means isolating \\\\( x \\\\) on one side of the equation. The initial equation is \\\\( 8x + 7 = -23 \\\\). To start, we need to eliminate the +7 from the left side.\",\"output\":\"8x + 7 = -23\"},{\"explanation\":\"Subtract 7 from both sides of the equation to move it to the other side. This helps in isolating the term with \\\\( x \\\\).\",\"output\":\"8x + 7 - 7 = -23 - 7\"},{\"explanation\":\"Simplifying both sides after subtracting 7. On the left side, \\\\( +7 \\\\) and \\\\( -7 \\\\) cancel each other out, leaving \\\\( 8x \\\\). On the right side, \\\\(-23 - 7\\\\) simplifies to \\\\(-30\\\\).\",\"output\":\"8x = -30\"},{\"explanation\":\"Now, we need to get \\\\( x \\\\) by itself. Since \\\\( x \\\\

In [7]:
print_pretty(math_reasoning.parsed.dict())

{
    "final_answer": "x = -\\frac{15}{4}",
    "steps": [
        {
            "explanation": "The goal is to solve for \\( x \\), which means isolating \\( x \\) on one side of the equation. The initial equation is \\( 8x + 7 = -23 \\). To start, we need to eliminate the +7 from the left side.",
            "output": "8x + 7 = -23"
        },
        {
            "explanation": "Subtract 7 from both sides of the equation to move it to the other side. This helps in isolating the term with \\( x \\).",
            "output": "8x + 7 - 7 = -23 - 7"
        },
        {
            "explanation": "Simplifying both sides after subtracting 7. On the left side, \\( +7 \\) and \\( -7 \\) cancel each other out, leaving \\( 8x \\). On the right side, \\(-23 - 7\\) simplifies to \\(-30\\).",
            "output": "8x = -30"
        },
        {
            "explanation": "Now, we need to get \\( x \\) by itself. Since \\( x \\) is being multiplied by 8, we should divide both sides by 8 to solv

### Structured data extraction

In [9]:
class ResearchPaperExtraction(BaseModel):
    title: str
    authors: List[str]
    abstract: str
    keywords: List[str]

In [10]:
paper = """
Title: Graph-Based Recommendation Systems: A Survey

Authors: John Doe, Jane Smith
Abstract: Recommendation systems play a crucial role in modern applications, providing personalized content to users.
In recent years, graph-based methods have emerged as powerful tools for improving recommendation quality.
This survey explores state-of-the-art graph-based recommendation techniques, including graph neural networks (GNNs),
random walk-based approaches, and knowledge graphs. We compare various models, highlight key challenges,
and discuss future research directions in this evolving field.
"""

In [11]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are an expert at structured data extraction. You will be given unstructured text from a research paper and should convert it into the given structure."},
        {"role": "user", "content": f"{paper}"}
    ],
    response_format=ResearchPaperExtraction,
)

research_paper = completion.choices[0].message.parsed

In [12]:
print_pretty(research_paper.dict())

{
    "abstract": "Recommendation systems play a crucial role in modern applications, providing personalized content to users. In recent years, graph-based methods have emerged as powerful tools for improving recommendation quality. This survey explores state-of-the-art graph-based recommendation techniques, including graph neural networks (GNNs), random walk-based approaches, and knowledge graphs. We compare various models, highlight key challenges, and discuss future research directions in this evolving field.",
    "authors": [
        "John Doe",
        "Jane Smith"
    ],
    "keywords": [],
    "title": "Graph-Based Recommendation Systems: A Survey"
}


### UI Generation

In [14]:
class UIType(str, Enum):
    div = "div"
    button = "button"
    header = "header"
    section = "section"
    field = "field"
    form = "form"

class Attribute(BaseModel):
    name: str
    value: str

class UI(BaseModel):
    type: UIType
    label: str
    children: List["UI"] 
    attributes: List[Attribute]

UI.model_rebuild() # This is required to enable recursive types

class Response(BaseModel):
    ui: UI


In [15]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a UI generator AI. Convert the user input into a UI."},
        {"role": "user", "content": "Make a User Profile Form"}
    ],
    response_format=Response,
)

ui = completion.choices[0].message.parsed
print_pretty(ui.dict())

{
    "ui": {
        "attributes": [
            {
                "name": "action",
                "value": "/submit_profile"
            },
            {
                "name": "method",
                "value": "post"
            }
        ],
        "children": [
            {
                "attributes": [
                    {
                        "name": "type",
                        "value": "text"
                    },
                    {
                        "name": "name",
                        "value": "first_name"
                    }
                ],
                "children": [],
                "label": "First Name",
                "type": "field"
            },
            {
                "attributes": [
                    {
                        "name": "type",
                        "value": "text"
                    },
                    {
                        "name": "name",
                        "value": "last_name"
             

### Moderation

In [16]:
class Category(str, Enum):
    violence = "violence"
    sexual = "sexual"
    self_harm = "self_harm"

class ContentCompliance(BaseModel):
    is_violating: bool
    category: Optional[Category]
    explanation_if_violating: Optional[str]


In [17]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Determine if the user input violates specific guidelines and explain if they do."},
        {"role": "user", "content": "How do I prepare for a job interview?"}
    ],
    response_format=ContentCompliance,
)

compliance = completion.choices[0].message.parsed

In [18]:
compliance

ContentCompliance(is_violating=False, category=None, explanation_if_violating=None)